In [3]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

from scipy.stats import chi2_contingency
from sklearn.feature_extraction.text import CountVectorizer

import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv("/content/world_bank_preprocessed.csv", encoding="latin1")
df.head()

,Project ID,Region,Country,Project Status,Last Stage Reached Name,Project Name,Project Development Objective,Implementing Agency,Consultant Services Required,Project URL,...,Grant Amount,Borrower,Lending Instrument,Environmental Assessment Category,Environmental and Social Risk,Sector 1,Sector 2,Sector 3,Theme 1,Theme 2
0,P006578,Latin America and Caribbean,Republic of Chile,Closed,unknown,Power and Irrigation Project,Objective to be Determined.,Ministry of Energy,Y,http://projects.worldbank.org/P006578/null?lan...,...,3500000.0,Ministry of Finance,Specific Investment Loan,C,Substantial,(Historic)Hydro,Central Government (Central Agencies),Central Government (Central Agencies),Administrative and civil service reform,Climate change
1,P006577,Latin America and Caribbean,Republic of Chile,Closed,Concept Review,Agricultural Machinery Project,Objective to be Determined.,Ministry of Energy,Y,http://projects.worldbank.org/P006577/null?lan...,...,3500000.0,Ministry of Finance,Financial Intermediary Loan,C,Substantial,(Historic)Agriculture adjustment,Central Government (Central Agencies),Central Government (Central Agencies),Administrative and civil service reform,Climate change
2,P007497,Latin America and Caribbean,United Mexican States,Closed,Begin Appraisal,Electric Power Development Project,Objective to be Determined.,Secretaria de Educacion Publica,Y,http://projects.worldbank.org/P007497/null?lan...,...,0.0,Ministry of Finance,Sector Investment and Maintenance Loan,C,Moderate,(Historic)Electric power and other energy adju...,Central Government (Central Agencies),Central Government (Central Agencies),Education for all,Climate change
3,P007498,Latin America and Caribbean,United Mexican States,Closed,Begin Appraisal,Electric Power Development Project,Objective to be Determined.,Secretaria de Educacion Publica,Y,http://projects.worldbank.org/P007498/null?lan...,...,0.0,Ministry of Finance,Sector Investment and Maintenance Loan,C,Moderate,(Historic)Electric power and other energy adju...,Central Government (Central Agencies),Central Government (Central Agencies),Education for all,Climate change
4,P006214,Latin America and Caribbean,Federative Republic of Brazil,Closed,unknown,Power and Telephone Project,Objective to be Determined.,Ministry of Environment,Y,http://projects.worldbank.org/P006214/null?lan...,...,0.0,Ministry of Finance,Sector Investment and Maintenance Loan,B,Moderate,(Historic)Thermal,Sub-National Government,Sub-National Government,Biodiversity,Rural services and infrastructure


In [5]:
df.shape

(20130, 27)

In [6]:
df.columns

Index(['Project ID', 'Region', 'Country', 'Project Status',
       'Last Stage Reached Name', 'Project Name',
       'Project Development Objective ', 'Implementing Agency',
       'Consultant Services Required', 'Project URL', 'Board Approval Date',
       'Project Closing Date', 'Financing Type', 'Current Project Cost',
       'IBRD Commitment ', 'IDA Commitment', 'Total IDA and IBRD Commitment',
       'Grant Amount', 'Borrower', 'Lending Instrument',
       'Environmental Assessment Category', 'Environmental and Social Risk',
       'Sector 1', 'Sector 2', 'Sector 3', 'Theme 1', 'Theme 2'],
      dtype='object')

 **Exploratory Data Analysis**

**How many projects are there in each region and country?**

In [7]:
df1 = df[['Region', 'Country', 'Project Status','Project Name']]
df1.head()

,Region,Country,Project Status,Project Name
0,Latin America and Caribbean,Republic of Chile,Closed,Power and Irrigation Project
1,Latin America and Caribbean,Republic of Chile,Closed,Agricultural Machinery Project
2,Latin America and Caribbean,United Mexican States,Closed,Electric Power Development Project
3,Latin America and Caribbean,United Mexican States,Closed,Electric Power Development Project
4,Latin America and Caribbean,Federative Republic of Brazil,Closed,Power and Telephone Project


In [8]:
df1_res = df1.groupby(["Country", "Region"][::-1]).agg({"Project Name" : ["count"]})
df1_res.columns = [("_").join(col).strip() for col in df1_res.columns]
df1_res

Project Name_count
Region                     Country                                               
East Asia and Pacific      Democratic Republic of Timor-Leste                  57
                           Federated States of Micronesia                      14
                           Kingdom of Cambodia                                124
                           Kingdom of Thailand                                192
                           Kingdom of Tonga                                    42
...                                                                           ...
Western and Central Africa Republic of Senegal                                233
                           Republic of Sierra Leone                           160
                           Republic of The Gambia                              86
                           Republic of Togo                                   123
                           Western and Central Africa                         155

[137 rows x 1 columns]

In [9]:
df1_res = df1_res.sort_values(by="Project Name_count", ascending=False)
df1_res.reset_index(inplace=True)

In [10]:
df1_res.columns

Index(['Region', 'Country', 'Project Name_count'], dtype='object')

In [11]:
for i in df1_res.Region.value_counts().keys():
    df_temp = df1_res[df1_res["Region"] == i]
    fig = px.bar(
        df_temp,
        x="Country",
        y="Project Name_count",
        color="Country",
    )

    fig.update_layout(
        title=f"Number of projects in {i}",
        xaxis_tickangle=-45,
        xaxis_title = "Country",
        yaxis_title = "Number of Projects"
    )
    fig.show()

**What is the distribution of project status (e.g. active, closed) across regions and countries?**

In [12]:
df.columns

Index(['Project ID', 'Region', 'Country', 'Project Status',
       'Last Stage Reached Name', 'Project Name',
       'Project Development Objective ', 'Implementing Agency',
       'Consultant Services Required', 'Project URL', 'Board Approval Date',
       'Project Closing Date', 'Financing Type', 'Current Project Cost',
       'IBRD Commitment ', 'IDA Commitment', 'Total IDA and IBRD Commitment',
       'Grant Amount', 'Borrower', 'Lending Instrument',
       'Environmental Assessment Category', 'Environmental and Social Risk',
       'Sector 1', 'Sector 2', 'Sector 3', 'Theme 1', 'Theme 2'],
      dtype='object')

In [13]:
df2 = df[["Project Status", "Country", "Region"][::-1]]
df2.head()

,Region,Country,Project Status
0,Latin America and Caribbean,Republic of Chile,Closed
1,Latin America and Caribbean,Republic of Chile,Closed
2,Latin America and Caribbean,United Mexican States,Closed
3,Latin America and Caribbean,United Mexican States,Closed
4,Latin America and Caribbean,Federative Republic of Brazil,Closed


In [14]:
df2_res = pd.crosstab([df2.Region, df2.Country], df2["Project Status"])
df2_res.reset_index(inplace=True)
df2_res

Project Status,Region,Country,Active,Closed,Dropped,Pipeline
0,East Asia and Pacific,Democratic Republic of Timor-Leste,8,46,2,1
1,East Asia and Pacific,Federated States of Micronesia,11,2,0,1
2,East Asia and Pacific,Kingdom of Cambodia,28,68,18,10
3,East Asia and Pacific,Kingdom of Thailand,1,165,23,3
4,East Asia and Pacific,Kingdom of Tonga,15,24,2,1
...,...,...,...,...,...,...
132,Western and Central Africa,Republic of Senegal,27,188,11,7
133,Western and Central Africa,Republic of Sierra Leone,29,116,8,7
134,Western and Central Africa,Republic of The Gambia,16,59,7,4
135,Western and Central Africa,Republic of Togo,12,94,9,8


In [15]:
for i in df2_res.Region.value_counts().keys():
    df_temp = df2_res[df2_res["Region"] == i]
    fig = px.bar(
        df_temp,
        x="Country",
        y=["Active", "Closed", "Dropped", "Pipeline"],
    )

    fig.update_layout(
        title=f"Project Status in {i}",
        xaxis_tickangle=-45,
        xaxis_title = "Country",
        yaxis_title = "Project Status"
    )
    fig.show()

**Which countries have the highest and lowest number of projects?**

In [16]:
df3 = df[["Country", "Project ID"]]
df3.head()

,Country,Project ID
0,Republic of Chile,P006578
1,Republic of Chile,P006577
2,United Mexican States,P007497
3,United Mexican States,P007498
4,Federative Republic of Brazil,P006214


In [17]:
df3_res = df3.groupby("Country").agg({"Project ID" : ["count"]})
df3_res.columns = [("_").join(col).strip() for col in df3_res.columns]
df3_res.sort_values(by="Project ID_count", inplace=True, ascending=False)

In [18]:
fig = px.bar(
    df3_res,
    x=df3_res.index,
    y="Project ID_count",
    color=df3_res.index
)
fig.update_layout(
    xaxis_tickangle = -45,
    xaxis_title = "Countries",
    yaxis_title = "Number of Projects"
)
fig.show()

In [19]:
top10 = df3_res.head(10).reset_index()
bottom10 = df3_res.tail(10).reset_index()

In [20]:
top10.head()

,Country,Project ID_count
0,Republic of India,927
1,Republic of Indonesia,653
2,People's Republic of China,610
3,Federative Republic of Brazil,584
4,Islamic Republic of Pakistan,503


In [21]:
bottom10.head()

,Country,Project ID_count
0,South Asia,24
1,Commonwealth of Dominica,22
2,St. Vincent and the Grenadines,21
3,Republic of the Marshall Islands,21
4,Federated States of Micronesia,14


In [22]:
fig1 = px.bar(
    top10,
    x="Country",
    y="Project ID_count",
    color="Country",
    title="Top 10 Countries with highest number of Projects"
)

fig2 = px.bar(
    bottom10,
    x="Country",
    y="Project ID_count",
    color="Country",
    title="Bottom 10 Countries with Lowest number of Projects"
)

In [23]:
for i in [fig1, fig2]:
    i.show()

**What is the distribution of project costs? How much funding has been committed by IBRD and IDA? Are there any differences in funding between regions/countries or project statuses?**

In [24]:
df.columns

Index(['Project ID', 'Region', 'Country', 'Project Status',
       'Last Stage Reached Name', 'Project Name',
       'Project Development Objective ', 'Implementing Agency',
       'Consultant Services Required', 'Project URL', 'Board Approval Date',
       'Project Closing Date', 'Financing Type', 'Current Project Cost',
       'IBRD Commitment ', 'IDA Commitment', 'Total IDA and IBRD Commitment',
       'Grant Amount', 'Borrower', 'Lending Instrument',
       'Environmental Assessment Category', 'Environmental and Social Risk',
       'Sector 1', 'Sector 2', 'Sector 3', 'Theme 1', 'Theme 2'],
      dtype='object')

In [25]:
df4 = df[["Current Project Cost"]]
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20130 entries, 0 to 20129
Data columns (total 1 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Current Project Cost  20130 non-null  float64
dtypes: float64(1)
memory usage: 157.4 KB


In [26]:
df4 = df4.dropna()

In [27]:
px.histogram(
    df4,
    title="Distribution of Project Costs"
)

In [28]:
df5 = df[['Region', 'Country','IBRD Commitment ','IDA Commitment']]
df5.head()

,Region,Country,IBRD Commitment,IDA Commitment
0,Latin America and Caribbean,Republic of Chile,13500000.0,0.0
1,Latin America and Caribbean,Republic of Chile,2500000.0,0.0
2,Latin America and Caribbean,United Mexican States,24100000.0,0.0
3,Latin America and Caribbean,United Mexican States,10000000.0,0.0
4,Latin America and Caribbean,Federative Republic of Brazil,75000000.0,0.0


In [29]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20130 entries, 0 to 20129
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Region            20130 non-null  object 
 1   Country           20130 non-null  object 
 2   IBRD Commitment   20130 non-null  float64
 3   IDA Commitment    20130 non-null  float64
dtypes: float64(2), object(2)
memory usage: 629.2+ KB


In [30]:
df5_res = df5.groupby(['Region', 'Country']).agg({
    "IBRD Commitment " : ["mean"],
    "IDA Commitment" : ["mean"]
})

df5_res.columns = [("_").join(col).strip() for col in df5_res.columns]
df5_res.sort_values(by=["IBRD Commitment _mean", "IDA Commitment_mean"], ascending=False, inplace=True)
df5_res.reset_index(inplace=True)
df5_res

,Region,Country,IBRD Commitment _mean,IDA Commitment_mean
0,Europe and Central Asia,Republic of Turkiye,1.880451e+08,5.871711e+05
1,Latin America and Caribbean,United Mexican States,1.819801e+08,0.000000e+00
2,Latin America and Caribbean,Argentine Republic,1.731380e+08,0.000000e+00
3,Europe and Central Asia,Republic of Kazakhstan,1.647131e+08,0.000000e+00
4,Europe and Central Asia,Ukraine,1.585475e+08,7.042254e+06
...,...,...,...,...
132,Latin America and Caribbean,Central America,0.000000e+00,6.153846e+05
133,Latin America and Caribbean,Latin America,0.000000e+00,0.000000e+00
134,Middle East and North Africa,West Bank and Gaza,0.000000e+00,0.000000e+00
135,Other,Multi-Regional,0.000000e+00,0.000000e+00


In [31]:
for i in df5_res.Region.value_counts().keys():
    df_temp = df5_res[df5_res["Region"] == i]

    fig = px.bar(
        df_temp,
        x="Country",
        y=["IBRD Commitment _mean","IDA Commitment_mean"],
        color_discrete_map={"IBRD Commitment _mean" : "orange", "IDA Commitment_mean" : "green"},
        barmode='group',
        title=f"Average Commitment by IBRD and IDA in {i}"
    )
    fig.update_layout(
        xaxis_tickangle=-45,
        xaxis_title="Country",
        yaxis_title="Average Commitment"
    )
    fig.show()

**TAIYO Requirements**

**1. Extract entities
Use Named Entity Recognition (NER) to identify and extract sector, sub-sector, location, or entities like Government Agency, Company Name, Contractors, Investor, or unit measurements such as cost per square kilometer. Ideally using the projects / tenders description and the original PDF document**

**Please refer NER.py**

**2. Similar Projects**

**Word2vec and / or cosine similarity for semantic and syntactic for identifying similar projects. For example: For a given project identify all similar projects within the past 10 years within 500 miles**

**The Approach:**

**1. For finding the similar projects, our target feature is the "Project Name". We will first find the important features for evaluating the target and we will achieve that using the chi-square statistical hypothesis test.**

**2. For simplicity, we will just use the top 3 features. Once we have those features, we will Feature Engineer a new column which will be a concatenation of these features.**

**3. Next, we will create a TfIdfVectorizer and pass this feature in the function before calculating the cosine similarity.**

**4. Once we have the similarity martix, we can take a Project Name as an input and compare it to all the entries and find the most similar Projects (Top 10)**

**5. As the task is to find the similar projects within the past 10 years we will make sure our data is within that threshold**

In [32]:
df.columns

Index(['Project ID', 'Region', 'Country', 'Project Status',
       'Last Stage Reached Name', 'Project Name',
       'Project Development Objective ', 'Implementing Agency',
       'Consultant Services Required', 'Project URL', 'Board Approval Date',
       'Project Closing Date', 'Financing Type', 'Current Project Cost',
       'IBRD Commitment ', 'IDA Commitment', 'Total IDA and IBRD Commitment',
       'Grant Amount', 'Borrower', 'Lending Instrument',
       'Environmental Assessment Category', 'Environmental and Social Risk',
       'Sector 1', 'Sector 2', 'Sector 3', 'Theme 1', 'Theme 2'],
      dtype='object')

In [33]:
df["Board Approval Date"] = pd.to_datetime(df["Board Approval Date"])
df["Project Closing Date"] = pd.to_datetime(df["Project Closing Date"])

df["Project Closing Date"].sort_values(ascending=False).reset_index(drop=True)[0]

Timestamp('2037-08-26 00:00:00')

In [34]:
categorical_columns = list(df.select_dtypes('object').columns)
categorical_columns

['Project ID',
 'Region',
 'Country',
 'Project Status',
 'Last Stage Reached Name',
 'Project Name',
 'Project Development Objective ',
 'Implementing Agency',
 'Consultant Services Required',
 'Project URL',
 'Financing Type',
 'Borrower',
 'Lending Instrument',
 'Environmental Assessment Category',
 'Environmental and Social Risk',
 'Sector 1',
 'Sector 2',
 'Sector 3',
 'Theme 1',
 'Theme 2']

In [35]:
df_sim = df[categorical_columns]
df_sim["Board Approval Date"] = df["Board Approval Date"]
df_sim = df_sim[df_sim["Board Approval Date"]>="2013, 2, 28"]

df_sim.reset_index(inplace=True, drop=True)
df_sim.shape

(5754, 21)

In [36]:

df_sim["Board Approval Date"].max().date()

datetime.date(2023, 2, 28)

In [37]:
df_sim["Board Approval Date"].min().date()

datetime.date(2013, 2, 28)

In [38]:
modes = df.groupby("Country")["Implementing Agency", "Borrower"].apply(lambda x: x.mode()).reset_index()
modes = modes.loc[modes["level_1"] == 0]
modes.drop("level_1", inplace=True, axis=1)
modes.dropna(inplace=True)
modes.reset_index(drop=True, inplace=True)
def fill_IA(country):
    return modes.loc[modes["Country"] == country]["Implementing Agency"][0]

def fill_B(country):
    return modes.loc[modes["Country"] == country]["Borrower"][0]
count = 0
try:
    df_sim["Implementing Agency"] = df_sim.apply(lambda row: fill_IA(row["Country"]) if pd.isnull(row["Implementing Agency"]) else row["Implementing Agency"], axis=1)
    df_sim["Borrower"] = df_sim.apply(lambda row: fill_B(row["Country"]) if pd.isnull(row["Borrower"]) else row["Borrower"], axis=1)
except KeyError:
    pass

In [39]:
p_values = {}
for col in df_sim.columns:
    if col == 'Project Name':
        continue
    contingency_table = pd.crosstab(df_sim['Project Name'], df_sim[col])
    chi2, pval, _, _ = chi2_contingency(contingency_table)
    p_values[col] = pval

top_3_features = sorted(p_values.items(), key=lambda x: x[1])[:4]

print("Top 3 most correlated features with 'Project Name':")
for feature, p_value in top_3_features:
    print(f"{feature}: {p_value}")

Top 3 most correlated features with 'Project Name':
Project Development Objective : 3.1168391837413667e-170
Implementing Agency: 6.186132175436858e-52
Board Approval Date: 1.3165249472511939e-27
Sector 1: 0.00027213320754475995


**From the CHI squared statistical test we found 3 features to be the most correlated with our target "Project Name"**

**1. "Project Development Objective"**

**2. "Implementing Agency"**

**3. "Borrower"**

In [40]:
df_sim = df_sim[["Project Name", "Project Development Objective ", "Implementing Agency", "Sector 1"]].reset_index(drop=True)
df_sim.head()

,Project Name,Project Development Objective,Implementing Agency,Sector 1
0,Vietnam School Readiness Promotion Project,The objective of the Vietnam School Readiness ...,Ministry of Education and Training,Early Childhood Education
1,China:Fujian Meizhou Bay Navigation Improvemen...,To improve the capacity of the main navigation...,Fujian Provincial Department of Transportation,Ports/Waterways
2,Health Sustainable Development Goals Program-f...,To improve the delivery and use of a comprehen...,"Ministry of Health (MOH), Immigration and Cit...",Health
3,Higher Education Development Policy Program - ...,"HEDPO3 aims to strengthen governance, financin...",Ministry of Education and Training,Tertiary Education
4,Afghanistan: System Enhancement for Health Act...,The project development objectives are to expa...,Ministry of Public Health,Health


In [41]:
df_sim.shape

(5754, 4)

In [42]:
df_sim["important"] = df_sim["Project Name"] + " " + df_sim["Project Development Objective "] + " " + df_sim["Implementing Agency"]+ " " + df_sim["Sector 1"]
df_sim["ids"] = [i for i in range(0, df_sim.shape[0])]
df_sim.head()

,Project Name,Project Development Objective,Implementing Agency,Sector 1,important,ids
0,Vietnam School Readiness Promotion Project,The objective of the Vietnam School Readiness ...,Ministry of Education and Training,Early Childhood Education,Vietnam School Readiness Promotion Project The...,0
1,China:Fujian Meizhou Bay Navigation Improvemen...,To improve the capacity of the main navigation...,Fujian Provincial Department of Transportation,Ports/Waterways,China:Fujian Meizhou Bay Navigation Improvemen...,1
2,Health Sustainable Development Goals Program-f...,To improve the delivery and use of a comprehen...,"Ministry of Health (MOH), Immigration and Cit...",Health,Health Sustainable Development Goals Program-f...,2
3,Higher Education Development Policy Program - ...,"HEDPO3 aims to strengthen governance, financin...",Ministry of Education and Training,Tertiary Education,Higher Education Development Policy Program - ...,3
4,Afghanistan: System Enhancement for Health Act...,The project development objectives are to expa...,Ministry of Public Health,Health,Afghanistan: System Enhancement for Health Act...,4


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vec=TfidfVectorizer()
vecs = vec.fit_transform(df_sim["important"])
vecs.shape

(5754, 12769)

In [44]:
sim = cosine_similarity(vecs)
sim.shape

(5754, 5754)

In [45]:
def find_similar(project_name):
    name_id = df_sim[df_sim["Project Name"] == project_name]["ids"].values[0]
    scores = list(enumerate(sim[name_id]))
    sorted_scores = sorted(scores, key=lambda x : x[1], reverse=True)
    sorted_scores = sorted_scores[1:]
    projects = [df_sim[projects[0]==df_sim["ids"]]["Project Name"].values[0] for projects in sorted_scores]
    return projects
try:
    input_project = str(input("Enter a project name: "))

    if input_project not in df_sim['Project Name'].values:
        raise IndexError("Please enter a valid Project Name from the dataset!")

    print("\nHere are the top 10 similar projects within the past 10 years:\n")
    for i in range(10):
        print(f"{i+1}. {find_similar(input_project)[i]}")

except IndexError as e:
    print(e)

Enter a project name: Vietnam School Readiness Promotion Project

Here are the top 10 similar projects within the past 10 years:

1. Learning for the Future Project
2. Nepal: Additional Financing for School Sector Development Program
3. Nepal School Sector Development Program
4. School Sector Reform Program Additional Financing
5. Nepal: School Sector Reform Program Additional Financing
6. CoVID 19 School Sector Response (GPE)
7. Enhancing the Foundation of Learning
8. Getting to Best in Education
9. Cambodia Global Partnership for Education Second Education Support Project
10. Vietnam Scaling Up Energy Efficiency Guarantee


**Trends**

**Show data visualizations for aggregated time series, bar chart / line chart, where the X-axis is Time and Y-axis is ‘Total number of records’ and/or ‘Total budget/cost’. These charts can be segmented to show different countries or sectors within a country over time.**

In [46]:
df.columns

Index(['Project ID', 'Region', 'Country', 'Project Status',
       'Last Stage Reached Name', 'Project Name',
       'Project Development Objective ', 'Implementing Agency',
       'Consultant Services Required', 'Project URL', 'Board Approval Date',
       'Project Closing Date', 'Financing Type', 'Current Project Cost',
       'IBRD Commitment ', 'IDA Commitment', 'Total IDA and IBRD Commitment',
       'Grant Amount', 'Borrower', 'Lending Instrument',
       'Environmental Assessment Category', 'Environmental and Social Risk',
       'Sector 1', 'Sector 2', 'Sector 3', 'Theme 1', 'Theme 2'],
      dtype='object')

**How has the total project cost varied over time for each country in the given region?**

In [47]:
df6 =  df[["Region","Country", "Current Project Cost", "Board Approval Date"]]
df6.head()

,Region,Country,Current Project Cost,Board Approval Date
0,Latin America and Caribbean,Republic of Chile,13500000.0,1948-03-25
1,Latin America and Caribbean,Republic of Chile,2500000.0,1948-03-25
2,Latin America and Caribbean,United Mexican States,24100000.0,1949-01-06
3,Latin America and Caribbean,United Mexican States,10000000.0,1949-01-06
4,Latin America and Caribbean,Federative Republic of Brazil,75000000.0,1949-01-27


In [48]:
df6_res = df6.groupby(["Region", "Country", "Board Approval Date"]).agg({"Current Project Cost" : ["sum"]}).reset_index()
df6_res.columns = [("_").join(col) for col in df6_res.columns]
df6_res.sort_values(ascending=False, by="Board Approval Date_")
df6_res.head()

,Region_,Country_,Board Approval Date_,Current Project Cost_sum
0,East Asia and Pacific,Democratic Republic of Timor-Leste,2000-02-21,75500000.0
1,East Asia and Pacific,Democratic Republic of Timor-Leste,2000-03-31,500000.0
2,East Asia and Pacific,Democratic Republic of Timor-Leste,2000-04-11,4850000.0
3,East Asia and Pacific,Democratic Republic of Timor-Leste,2000-04-19,29760000.0
4,East Asia and Pacific,Democratic Republic of Timor-Leste,2000-06-02,12700000.0


In [49]:
for i in df6_res.Region_.value_counts().keys():
    df_temp = df6_res[df6_res["Region_"] == i]
    fig = px.line(
        df_temp,
        x='Board Approval Date_',
        y='Current Project Cost_sum',
        color='Country_',
        title=f'Total Project Cost over Time for each Country in {i}'
    )
    fig.show()

**How has the number of projects approved by the board changed over time for different sectors and countries?**

In [50]:
df7 = df[["Country", "Project ID", "Sector 1", "Board Approval Date"]]
df7.head()

,Country,Project ID,Sector 1,Board Approval Date
0,Republic of Chile,P006578,(Historic)Hydro,1948-03-25
1,Republic of Chile,P006577,(Historic)Agriculture adjustment,1948-03-25
2,United Mexican States,P007497,(Historic)Electric power and other energy adju...,1949-01-06
3,United Mexican States,P007498,(Historic)Electric power and other energy adju...,1949-01-06
4,Federative Republic of Brazil,P006214,(Historic)Thermal,1949-01-27


In [51]:
df7['Year'] = df['Board Approval Date'].dt.year
grouped_df = df7.groupby(['Country', 'Year'])['Project ID'].count().reset_index()
fig = px.line(grouped_df, x='Year', y='Project ID', color='Country')
fig.show()

**What is the trend in the total project cost for projects with different environmental and social risk levels?**

In [52]:
fig = px.line(df, x='Board Approval Date', y='Current Project Cost', color='Environmental and Social Risk',
              line_group='Environmental and Social Risk',
              hover_name='Environmental and Social Risk',
              labels={'Current Project Cost': 'Total Project Cost', 'Environmental and Social Risk': 'Risk Level',
                      'Board Approval Date': 'Approval Date'},
              title='Trend in Total Project Cost by Environmental and Social Risk Level')

fig.show()

**How has the environmental assessment category for projects changed over time?**

In [53]:
env = df.copy()
env['Year'] = env['Board Approval Date'].dt.year

pivot_df = pd.pivot_table(env, values='Project ID', index='Year', columns='Environmental Assessment Category', aggfunc='count', fill_value=0)

fig = px.line(pivot_df, x=pivot_df.index, y=pivot_df.columns, title='Environmental Assessment Category by Year')
fig.show()
df["Lending Instrument"].value_counts().keys()

Index(['Specific Investment Loan', 'Investment Project Financing',
       'Technical Assistance Loan', 'Development Policy Lending',
       'Sector Investment and Maintenance Loan', 'Financial Intermediary Loan',
       'Emergency Recovery Loan', 'Structural Adjustment Loan',
       'Adaptable Program Loan', 'Program-for-Results Financing',
       'Sector Adjustment Loan', 'Learning and Innovation Loan',
       'Programmatic Structural Adjustment Loan', 'UNIDENTIFIED',
       'Poverty Reduction Support Credit', 'Rehabilitation Loan',
       'Debt and Debt Service Reduction Loan',
       'Special Structural Adjustment Loan'],
      dtype='object')

**How has the use of different lending instruments for project financing changed over time for the World Bank Group?**

In [54]:
df_lending = df.groupby(['Lending Instrument', 'Board Approval Date']).size().reset_index(name='count')
fig = px.line(df_lending, x="Board Approval Date", y="count", color="Lending Instrument",
              title="Use of Different Lending Instruments over Time")
fig.show()

**GPT**

In [55]:
import pandas as pd
import json

data = pd.read_csv("/content/world_bank_preprocessed.csv")

def generate_completion(row):
    summary = f"The project '{row['Project Name']}' is a {row['Project Development Objective ']} project in {row['Country']}. It was approved by the World Bank on {row['Board Approval Date']} and was closed on {row['Project Closing Date']}. The total cost of the project was {row['Current Project Cost']} USD, with an IBRD commitment of {row['IBRD Commitment ']} USD and an IDA commitment of {row['IDA Commitment']} USD."

    return summary

pairs = []

for index, row in data.iterrows():
    prompt = f"What is the summary of the '{row['Project Name']}' in {row['Country']}?"
    completion = generate_completion(row)
    pair = {"prompt": prompt, "completion": completion}
    pairs.append(pair)

with open("prompt_completion_pairs.json", "w") as outfile:
    json.dump(pairs, outfile)


**CREDENTIALS**





In [56]:
pip install credentials


In [57]:
from credentials import API_KEY
import openai, pandas as pd


openai.api_key = API_KEY
model_id = "ft-CTyrZcYHx15C0kddVRUojf2d"


df = pd.read_csv("/content/world_bank_preprocessed.csv")
p_names = df["Project Name"].value_counts().keys().to_list()
c_names = df["Country"].value_counts().keys().to_list()

def validate_and_preprocess(project, country):

    matches = [name for name in c_names if country.lower() in name.lower()]

    if ((project in p_names) and (len(matches) != 0)):
        return project, matches[0]
    else:
        return 0, 0

while 1:

    print("What Project would you like to know about?\n")

    p_name = input("Project Name (Please type in the exact project name) : ")
    c_name = input("Country: ")

    project, country = validate_and_preprocess(p_name, c_name)

    if ((project, country) != (0,0)):

        prompt = f"What is the summary of the {project} in {country}?"

        response = openai.Completion.create(
            engine=model_id,
            prompt=prompt,
            max_tokens=100,
            n=1,
            stop=None,
            temperature=0.5,
        )

        print(response.choices[0].text)

    else:
        print("Data does not exist!")


# openai api fine_tunes.follow -i ft-CTyrZcYHx15C0kddVRUojf2d

ImportError: cannot import name 'API_KEY' from 'credentials' (/usr/local/lib/python3.10/dist-packages/credentials/__init__.py)

**OBSERVER**

In [ ]:
import time, pandas as pd
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# def on_created(event):
#     print("Created")

# def on_deleted(event):
#     print("Deleted")

def on_modified(event):

    # Step 1: Pre-Process the new Data
    data = pd.read_csv("world_bank_preprocessed.csv")

# def on_moved(event):
#     print("Moved")

if __name__ == "__main__":

    event_handler = FileSystemEventHandler()

    event_handler.on_modified = on_modified

    path = r"C:\Users\DELL\Desktop\Taiyo-Testing"

    observer = Observer()
    observer.schedule(event_handler, path, recursive=True)

    observer.start()
    try:
        while True:
            print("Monitoring...")
            time.sleep(1)      # This increases computational costs but this can be fixed by using any cloud services like AWS
    except KeyboardInterrupt:
        observer.stop()
    observer.join()